# Imports 

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Dropout
from sklearn.model_selection import train_test_split
import sys

# Data 

In [4]:
months = {"JAN":1, "FEB":2, "MAR":3, "APR":4, "MAY":5, "JUN":6, 
          "JUL":7, "AUG":8, "SEP":9, "OCT":10, "NOV":11, "DEC":12}

data = pd.read_csv('MVA_Vehicle_Sales_Counts_by_Month_for_Calendar_Year_2002_-_2021_up_to_April.csv')
dataNew = pd.DataFrame(data)
dataNew = dataNew.drop(columns = ['Total Sales New', 'Total Sales Used'])


column = [
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 
    1, 2, 3, 4]
dataNew['Month'] = column

for i in range(0, 232):
    dataNew.iloc[i, 2] = dataNew.iloc[i, 2] + dataNew.iloc[i, 3]
dataNew = dataNew.drop(columns = ['Used', 'Month '])
dataNew = dataNew.rename(columns={'New': 'Total Sales'})

dataNew = dataNew[['Total Sales', 'Year ', 'Month']]
ind = pd.read_csv("historical_country_United_States_indicator_Consumer_Price_Index_CPI.csv")
ind["Date"] = pd.to_datetime(ind["DateTime"]).dt.date
ind = ind[ind["Date"]>=pd.to_datetime("2002-01-01")]
ind = ind[ind["Date"]<=pd.to_datetime("2021-05-01")]
dataNew = dataNew.join(ind["Value"].reset_index()).drop(columns=["index"])
ind = pd.read_csv("historical_country_United_States_indicator_Capacity_Utilization.csv")
ind["Date"] = pd.to_datetime(ind["DateTime"]).dt.date
ind = ind[ind["Date"]>=pd.to_datetime("2002-01-01")]
ind = ind[ind["Date"]<=pd.to_datetime("2021-05-01")]
ind = ind.rename(columns={'Value':'Value2'})
dataNew = dataNew.join(ind["Value2"].reset_index()).drop(columns=["index"])
#What are we testing with?
testData = dataNew.loc[dataNew['Year '] == 2019]
testData = testData.set_index([pd.Index(list(range(12)))])
anotherData = dataNew.set_index('Year ')
trainData = anotherData.drop(2019, axis = 0)
trainData = trainData.reset_index()
trainData = trainData[['Total Sales', 'Year ', 'Month', 'Value','Value2']]
dates = trainData.index


#print("dates: ", dates)



# Now we isolate training and test
#X = trainData.iloc[:, 0:4]
X = trainData.iloc[:, 0:4]
Y = trainData.iloc[:, 0]
#Xi_Test = testData.iloc[:, 0:4]
#Xi_Test = testData.iloc[:, 0:5]
#Yi_Test = testData.iloc[:, 0]

# Model

In [5]:
# Model Function
XTrain, XTest, YTrain, YTest = train_test_split(X, Y, test_size = 0.2, shuffle = True)
model = Sequential()
model.add(Dense(500, activation = 'relu', input_dim=len(X.columns))) # 4 with 1 indicator
model.add(Dense(250, activation = 'relu'))
model.add(Dropout(0.2))
#model.add(Dense(110, activation = 'relu'))
#model.add(Dropout(0.2))
model.add(Dense(1, activation = 'linear'))

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(XTrain, YTrain, epochs=1000, batch_size=32)

#results from model
loss = model.evaluate(XTest, YTest)
print('sqrt loss', np.sqrt(loss))
print('standard deviation', trainData['Total Sales'].std())
print(100*(trainData['Total Sales'].std()-np.sqrt(loss))/trainData['Total Sales'].std())

predictions = model.predict(X)

predictions_list = map(lambda x: x[0], predictions)
predictions_series = pd.Series(predictions_list)
dates_series = pd.Series(dates)

#Predicted_sales = model.predict(Xi_Test)
#new_dates_series=pd.Series(Xi_Test.index)
#new_predictions_list = map(lambda x: x[0], Predicted_sales)
#new_predictions_series = pd.Series(new_predictions_list,index=new_dates_series)

#export to csv
#new_predictions_series.to_csv("predicted_saless.csv",header=False)

# Author: Maryland Gov
# https://catalog.data.gov/dataset?publisher=opendata.maryland.gov&organization=state-of-maryland
# Used and New Car Data sales monthly
# https://catalog.data.gov/dataset/mva-vehicle-sales-counts-by-month-for-calendar-year-2002-2020-up-to-october

Train on 176 samples
Epoch 1/1000
176/176 [==============================] - 0s 2ms/sample - loss: 2843864756.3636
Epoch 2/1000
176/176 [==============================] - 0s 166us/sample - loss: 173737963.6364
Epoch 3/1000
176/176 [==============================] - 0s 168us/sample - loss: 458197186.9091
Epoch 4/1000
176/176 [==============================] - 0s 169us/sample - loss: 125681564.3636
Epoch 5/1000
176/176 [==============================] - 0s 170us/sample - loss: 85488326.5455
Epoch 6/1000
176/176 [==============================] - 0s 161us/sample - loss: 101426610.9091
Epoch 7/1000
176/176 [==============================] - 0s 166us/sample - loss: 35000337.0909
Epoch 8/1000
176/176 [==============================] - 0s 162us/sample - loss: 52090585.0909
Epoch 9/1000
176/176 [==============================] - 0s 170us/sample - loss: 41182884.7273
Epoch 10/1000
176/176 [==============================] - 0s 171us/sample - loss: 35264898.0000
Epoch 11/1000
176/176 [===========

176/176 [==============================] - 0s 166us/sample - loss: 28236639.2727
Epoch 86/1000
176/176 [==============================] - 0s 168us/sample - loss: 34637533.0909
Epoch 87/1000
176/176 [==============================] - 0s 155us/sample - loss: 30533632.0000
Epoch 88/1000
176/176 [==============================] - 0s 158us/sample - loss: 40159412.7273
Epoch 89/1000
176/176 [==============================] - 0s 163us/sample - loss: 36942582.1818
Epoch 90/1000
176/176 [==============================] - 0s 163us/sample - loss: 31724179.2727
Epoch 91/1000
176/176 [==============================] - 0s 164us/sample - loss: 29349198.0000
Epoch 92/1000
176/176 [==============================] - 0s 160us/sample - loss: 29385984.0000
Epoch 93/1000
176/176 [==============================] - 0s 157us/sample - loss: 29970586.5455
Epoch 94/1000
176/176 [==============================] - 0s 160us/sample - loss: 36027008.9091
Epoch 95/1000
176/176 [==============================] - 0s 161u

176/176 [==============================] - 0s 166us/sample - loss: 25122362.9091
Epoch 171/1000
176/176 [==============================] - 0s 176us/sample - loss: 29534881.8182
Epoch 172/1000
176/176 [==============================] - 0s 161us/sample - loss: 26837747.2727
Epoch 173/1000
176/176 [==============================] - 0s 169us/sample - loss: 26366706.3636
Epoch 174/1000
176/176 [==============================] - 0s 167us/sample - loss: 28830829.8182
Epoch 175/1000
176/176 [==============================] - 0s 158us/sample - loss: 30548032.5455
Epoch 176/1000
176/176 [==============================] - 0s 166us/sample - loss: 34645841.4545
Epoch 177/1000
176/176 [==============================] - 0s 164us/sample - loss: 36477343.6364
Epoch 178/1000
176/176 [==============================] - 0s 164us/sample - loss: 30038692.3636
Epoch 179/1000
176/176 [==============================] - 0s 160us/sample - loss: 32725138.9091
Epoch 180/1000
176/176 [==============================]

176/176 [==============================] - 0s 170us/sample - loss: 27430847.2727
Epoch 256/1000
176/176 [==============================] - 0s 171us/sample - loss: 30292794.7273
Epoch 257/1000
176/176 [==============================] - 0s 163us/sample - loss: 26652551.2727
Epoch 258/1000
176/176 [==============================] - 0s 175us/sample - loss: 30133039.6364
Epoch 259/1000
176/176 [==============================] - 0s 169us/sample - loss: 27245440.7273
Epoch 260/1000
176/176 [==============================] - 0s 165us/sample - loss: 27624113.0909
Epoch 261/1000
176/176 [==============================] - 0s 167us/sample - loss: 33715916.7273
Epoch 262/1000
176/176 [==============================] - 0s 164us/sample - loss: 29065201.0909
Epoch 263/1000
176/176 [==============================] - 0s 164us/sample - loss: 31156489.2727
Epoch 264/1000
176/176 [==============================] - 0s 173us/sample - loss: 33940811.0909
Epoch 265/1000
176/176 [==============================]

176/176 [==============================] - 0s 179us/sample - loss: 33930318.5455
Epoch 341/1000
176/176 [==============================] - 0s 175us/sample - loss: 30420222.9091
Epoch 342/1000
176/176 [==============================] - 0s 182us/sample - loss: 34992626.9091
Epoch 343/1000
176/176 [==============================] - 0s 177us/sample - loss: 34051216.3636
Epoch 344/1000
176/176 [==============================] - 0s 173us/sample - loss: 35521333.8182
Epoch 345/1000
176/176 [==============================] - 0s 174us/sample - loss: 37533040.3636
Epoch 346/1000
176/176 [==============================] - 0s 183us/sample - loss: 32293578.5455
Epoch 347/1000
176/176 [==============================] - 0s 178us/sample - loss: 30021099.6364
Epoch 348/1000
176/176 [==============================] - 0s 173us/sample - loss: 28043485.4545
Epoch 349/1000
176/176 [==============================] - 0s 183us/sample - loss: 31429025.8182
Epoch 350/1000
176/176 [==============================]

...

176/176 [==============================] - 0s 182us/sample - loss: 26120078.5455
Epoch 596/1000
176/176 [==============================] - 0s 185us/sample - loss: 25999960.9091
Epoch 597/1000
176/176 [==============================] - 0s 192us/sample - loss: 26215722.0000
Epoch 598/1000
176/176 [==============================] - 0s 187us/sample - loss: 24789228.0000
Epoch 599/1000
176/176 [==============================] - 0s 192us/sample - loss: 35042654.9091
Epoch 600/1000
176/176 [==============================] - 0s 194us/sample - loss: 32606243.6364
Epoch 601/1000
176/176 [==============================] - 0s 202us/sample - loss: 26845139.6364
Epoch 602/1000
176/176 [==============================] - 0s 190us/sample - loss: 24741134.3636
Epoch 603/1000
176/176 [==============================] - 0s 192us/sample - loss: 30816994.1818
Epoch 604/1000
176/176 [==============================] - 0s 184us/sample - loss: 28711365.2727
Epoch 605/1000
176/176 [==============================]

176/176 [==============================] - 0s 195us/sample - loss: 33868601.2727
Epoch 681/1000
176/176 [==============================] - 0s 194us/sample - loss: 24906142.0000
Epoch 682/1000
176/176 [==============================] - 0s 192us/sample - loss: 30254279.0909
Epoch 683/1000
176/176 [==============================] - 0s 180us/sample - loss: 31165240.7273
Epoch 684/1000
176/176 [==============================] - 0s 196us/sample - loss: 22463979.6364
Epoch 685/1000
176/176 [==============================] - 0s 183us/sample - loss: 27567937.0909
Epoch 686/1000
176/176 [==============================] - 0s 176us/sample - loss: 30432557.8182
Epoch 687/1000
176/176 [==============================] - 0s 178us/sample - loss: 30152106.1818
Epoch 688/1000
176/176 [==============================] - 0s 182us/sample - loss: 32173753.0909
Epoch 689/1000
176/176 [==============================] - 0s 186us/sample - loss: 29922525.0909
Epoch 690/1000
176/176 [==============================]

176/176 [==============================] - 0s 190us/sample - loss: 27804258.1818
Epoch 766/1000
176/176 [==============================] - 0s 189us/sample - loss: 30618772.1818
Epoch 767/1000
176/176 [==============================] - 0s 201us/sample - loss: 26344621.6364
Epoch 768/1000
176/176 [==============================] - 0s 189us/sample - loss: 30505759.4545
Epoch 769/1000
176/176 [==============================] - 0s 184us/sample - loss: 29463565.0909
Epoch 770/1000
176/176 [==============================] - 0s 186us/sample - loss: 29978130.3636
Epoch 771/1000
176/176 [==============================] - 0s 194us/sample - loss: 27510360.0000
Epoch 772/1000
176/176 [==============================] - 0s 189us/sample - loss: 30038585.0909
Epoch 773/1000
176/176 [==============================] - 0s 190us/sample - loss: 30676241.2727
Epoch 774/1000
176/176 [==============================] - 0s 188us/sample - loss: 27590788.0000
Epoch 775/1000
176/176 [==============================]

176/176 [==============================] - 0s 203us/sample - loss: 25865888.0000
Epoch 851/1000
176/176 [==============================] - 0s 202us/sample - loss: 28015731.6364
Epoch 852/1000
176/176 [==============================] - 0s 201us/sample - loss: 27765918.5455
Epoch 853/1000
176/176 [==============================] - 0s 194us/sample - loss: 29929505.0909
Epoch 854/1000
176/176 [==============================] - 0s 204us/sample - loss: 25706734.5455
Epoch 855/1000
176/176 [==============================] - 0s 207us/sample - loss: 31081648.0000
Epoch 856/1000
176/176 [==============================] - 0s 189us/sample - loss: 28247075.8182
Epoch 857/1000
176/176 [==============================] - 0s 190us/sample - loss: 23239048.0000
Epoch 858/1000
176/176 [==============================] - 0s 186us/sample - loss: 26807665.0909
Epoch 859/1000
176/176 [==============================] - 0s 190us/sample - loss: 26703330.7273
Epoch 860/1000
176/176 [==============================]

176/176 [==============================] - 0s 190us/sample - loss: 21619678.0000
Epoch 936/1000
176/176 [==============================] - 0s 197us/sample - loss: 27546366.1818
Epoch 937/1000
176/176 [==============================] - 0s 188us/sample - loss: 30079114.5455
Epoch 938/1000
176/176 [==============================] - 0s 192us/sample - loss: 25772379.0909
Epoch 939/1000
176/176 [==============================] - 0s 190us/sample - loss: 29007675.8182
Epoch 940/1000
176/176 [==============================] - 0s 189us/sample - loss: 29409584.0000
Epoch 941/1000
176/176 [==============================] - 0s 185us/sample - loss: 29925802.9091
Epoch 942/1000
176/176 [==============================] - 0s 185us/sample - loss: 21820432.0000
Epoch 943/1000
176/176 [==============================] - 0s 185us/sample - loss: 29023813.8182
Epoch 944/1000
176/176 [==============================] - 0s 198us/sample - loss: 31835814.0000
Epoch 945/1000
176/176 [==============================]